In [1]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

In [2]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go

In [3]:
df_params=pd.read_excel("params.xlsx")
df_location=pd.read_excel("location.xlsx")

In [4]:
# Basic data viz

df_params.columns

Index(['Unnamed: 0', 'Area', 'AreaRange', 'Area_from', 'Area_to', 'Build_year',
       'Building_floors_num', 'Building_material', 'Building_ownership',
       'Building_type', 'City', 'City_id', 'Construction_status', 'Country',
       'Equipment_types', 'Extras_types', 'Floor_no', 'Floors_num', 'Heating',
       'Id', 'MarketType', 'Media_types', 'OfferType', 'Offered_estates_type',
       'Photo', 'Price', 'PriceRange', 'Price_from', 'Price_per_m',
       'Price_per_m_from', 'Price_to', 'ProperType', 'Province', 'RegularUser',
       'Rent', 'Rooms_num', 'Security_types', 'State', 'Subregion', 'Title',
       'Windows_type', 'env', 'seller_id', 'user_type'],
      dtype='object')

In [5]:
df_params.shape

(12135, 44)

In [6]:
df_location.head()

,Unnamed: 0,Unnamed: 1,Id,address,city,district,latitude,longitude,radius,region,sub-region,zoom_level
0,0,NaN,60434297,"Warszawa, Wola, Młynów, ul. Leszno",Warszawa,Młynów,52.238238,20.978531,0,mazowieckie,Warszawa,16
1,1,NaN,60498475,"Warszawa, Białołęka",Warszawa,Białołęka,52.345895,20.943476,100,mazowieckie,Warszawa,16
2,2,NaN,60194413,"Warszawa, Praga-Południe",Warszawa,Praga-Południe,52.241680,21.084060,100,mazowieckie,Warszawa,12
3,3,NaN,59392882,"Warszawa, Mokotów, Górny Mokotów, Różana",Warszawa,Górny Mokotów,52.202488,21.017253,0,mazowieckie,Warszawa,15
4,4,NaN,60498567,"Warszawa, Żoliborz, Bitwy pod Rokitną",Warszawa,Żoliborz,52.262000,20.986000,0,mazowieckie,Warszawa,15


In [36]:
df_pricing=df_params[["Id","Area","Price",'Price_per_m']]

In [37]:
df=pd.merge(df_pricing,df_location[["Id","latitude","longitude","city","district"]],left_on="Id",right_on="Id")

In [38]:
df=df.query("city=='Warszawa'")
df=df.query("latitude>52 and latitude<53")
df=df.query("longitude>20 and longitude<21.2")

In [39]:
df.groupby("district", as_index=False).count()[["district","Id"]].sort_values(by="Id",ascending=False).head(10)

,district,Id
118,Wola,1239
50,Mokotów,1206
127,Śródmieście,906
4,Białołęka,903
74,Praga-Południe,620
3,Bemowo,565
110,Ursus,513
111,Ursynów,449
122,Włochy,425
116,Wilanów,415


In [40]:
data=[]
trace0=dict(
            lat=df.latitude,
            lon=df.longitude,
          
            type="scattermapbox",
            mode='markers',
            marker=dict(
                opacity=0.5, 
                size=10, 
                color=df.Price_per_m,
               
                colorscale="jet",
                cauto=False,
                cmax=20000,
                showscale=True),
                text=df.Price_per_m,
            
           
            )



data=[trace0]

layout = dict(
    
   
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken ="pk.eyJ1IjoiamFubWFqZXdza2kiLCJhIjoiY2p1bnRkN2JiMTFuczRjbXFuaDg0YWswZSJ9.LICCkkSfzPTXlNZ3MC-HHg",
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 52.2,
            lon = 21
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 10,
        # default map style
        style = 'light'
    )
)

figure=dict(
    data=data,
    layout=layout
    )
iplot(figure)

In [41]:
df.describe()

,Id,Area,Price,Price_per_m,latitude,longitude
count,1.198500e+04,11924.000000,1.192400e+04,11845.000000,11985.000000,11985.000000
mean,6.027407e+07,62.703986,7.038394e+05,11223.531954,52.229848,20.999387
std,8.609744e+05,96.577767,6.172397e+05,3860.556522,0.048995,0.064877
min,3.824426e+07,14.000000,0.000000e+00,95.000000,52.075370,20.849917
25%,6.034143e+07,42.000000,4.300000e+05,8944.000000,52.193464,20.951713
50%,6.041585e+07,55.365000,5.500000e+05,10550.000000,52.230653,20.998061
75%,6.046034e+07,70.432500,7.500000e+05,12500.000000,52.250650,21.044850
max,6.049868e+07,10000.000000,1.406240e+07,116038.000000,52.379218,21.199369


In [78]:
rounding_num=0.2
df["lon_mod"]=np.round(df.longitude/rounding_num,1)*rounding_num


df["lat_mod"]=np.round(df.latitude/rounding_num,1)*rounding_num


In [79]:
df_piv=df.pivot_table(index="lat_mod",columns="lon_mod",values="Price_per_m",aggfunc=np.mean)

In [80]:
arr=df_piv.values

In [81]:
df_piv.shape

(16, 19)

In [84]:
df_piv.columns.shape

(19,)

In [85]:
df_piv.index.shape

(16,)

In [90]:
fig = go.Figure(data=go.Heatmap(
                   z=df_piv.values,
                   y=df_piv.index,
                   x=df_piv.columns,
                 ))
fig.show()

In [91]:
trace0=go.Surface(
        z=df_piv.values,
        y=df_piv.index,
        x=df_piv.columns,
       connectgaps=True
          

        )

data=[trace0]

In [92]:
figure=go.Figure(
    data=data,
    layout=go.Layout(
      
        scene=dict(
            xaxis=dict(title="Longitude"),
            yaxis=dict(title="Latitude"),
            zaxis=dict(title="Price per m2")

           
                  ),
         title="Real estate price per m2 geo distribution"
    ))
iplot(figure)

In [89]:
fig = go.Figure(data =
    go.Contour(
        z=df_piv.values,
        y=df_piv.index,
        x=df_piv.columns,
        connectgaps=True
    ))
fig.show()